<a href="https://colab.research.google.com/github/afungue/DataExtraction/blob/main/f1-consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenF1 https://openf1.org/
OpenF1 es una API gratuita y de código abierto que proporciona datos en tiempo real e históricos de la Fórmula 1.

La API ofrece una gran cantidad de información, incluyendo tiempos por vuelta, telemetría de los autos, comunicaciones por radio, y más. Ya sea que estés buscando crear paneles interactivos, profundizar en el análisis de las carreras o incluso desarrollar objetos conectados que se iluminen cada vez que tu piloto favorito tome la delantera, OpenF1 lo hace posible.

Los datos se pueden acceder en formatos JSON o CSV, lo que la hace amigable tanto para desarrolladores como para personas sin experiencia técnica. Para comenzar rápidamente, puedes acceder a la API a través de tu navegador web. Se proporciona una URL de ejemplo para cada método como referencia.

In [ ]:
!pip install requests


# Sesiones disponibles en la base de OpenF1
2023-2024

In [79]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# -------------------------------------------------------
# Script para obtener y mostrar sesiones de carreras de F1
# desde la API OpenF1. Las sesiones se filtran y se
# ordenan por año y fecha de inicio. Las fechas se
# ajustan a la zona horaria GMT-3 (Argentina) y se
# presentan en un formato legible para el usuario.
# -------------------------------------------------------

# Define la URL de la API para obtener todas las sesiones
url = 'https://api.openf1.org/v1/sessions'
api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

# Configura la solicitud con la clave API en los encabezados
request = Request(url)
request.add_header('Authorization', f'Bearer {api_key}')

# Definir la zona horaria de Argentina (GMT-3)
tz = pytz.timezone('America/Argentina/Buenos_Aires')

try:
    # Realiza la solicitud a la API
    response = urlopen(request)
    data = response.read().decode('utf-8')

    # Cargar la respuesta como JSON
    data_sessions = json.loads(data)

    # Ordenar las sesiones por año (descendente) y date_start (descendente)
    sorted_sessions = sorted(data_sessions, key=lambda x: (x['year'], x['date_start']), reverse=True)

    # Imprimir la información de todas las sesiones en el formato solicitado
    last_year = None
    last_location = None

    # Definir el ancho fijo para las columnas (ajustar según el contenido)
    column_width = 30  # Ancho fijo para las columnas de sesión

    for session in sorted_sessions:  # Iterar sobre las sesiones ya ordenadas
        if (session['year'], session['location']) != (last_year, last_location):
            # Solo imprime el año y la localidad una vez
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00")).astimezone(tz)
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00")).astimezone(tz)

            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            print(f"{session['year']}, {session['location']} (Inicio: {formatted_start}, Fin: {formatted_end})")
            last_year = session['year']
            last_location = session['location']

        # Ajustar el formato para que "session_name" y "session_key" tengan un tamaño constante
        session_name = session['session_name'].ljust(column_width)  # Ajustar a la izquierda con ancho fijo
        session_key = session['session_key']  # No es necesario ajustar, es numérico

        print(f"    session_name: {session_name} session_key: {session_key}")

except Exception as e:
    print("An error occurred:", str(e))


2024, Austin (Inicio: 18 October 2024, 18:30, Fin: 18 October 2024, 19:14)
    session_name: Sprint Qualifying              session_key: 9612
    session_name: Practice 1                     session_key: 9607
2024, Marina Bay (Inicio: 22 September 2024, 09:00, Fin: 22 September 2024, 11:00)
    session_name: Race                           session_key: 9606
    session_name: Qualifying                     session_key: 9602
    session_name: Practice 3                     session_key: 9601
    session_name: Practice 2                     session_key: 9600
    session_name: Practice 1                     session_key: 9599
2024, Baku (Inicio: 15 September 2024, 08:00, Fin: 15 September 2024, 10:00)
    session_name: Race                           session_key: 9598
    session_name: Qualifying                     session_key: 9594
    session_name: Practice 3                     session_key: 9593
    session_name: Practice 2                     session_key: 9592
    session_name: Practice 1

# Sesiones por año


In [65]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# Función para obtener sesiones por año
def get_sessions_by_year(year):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Ordenar las sesiones por date_start de más nueva a más antigua
        data_sessions.sort(key=lambda x: x['date_start'], reverse=True)

        # Imprimir la información de las sesiones
        current_year = None
        current_location = None
        print(f"{'Año':<6} {'Localidad':<20} {'Nombre':<30} {'ID':<6} {'Fecha inicio':<25} {'Fecha fin':<25}")
        print("=" * 120)

        # Definir la zona horaria de Argentina (GMT-3)
        tz = pytz.timezone('America/Argentina/Buenos_Aires')

        for session in data_sessions:
            if session['year'] != current_year:
                current_year = session['year']
                print(f"{current_year:<6}")

            if session['location'] != current_location:
                current_location = session['location']
                print(f"  {current_location:<20}")

            # Convertir las fechas a tu zona horaria
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00"))
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00"))

            # Ajustar a la zona horaria GMT-3
            date_start = date_start.astimezone(tz)
            date_end = date_end.astimezone(tz)

            # Formatear las fechas
            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            # Imprimir nombre de sesión, ID y fechas con ajuste de espacio
            print(f"    {session['session_name']:<18} (ID: {session['session_key']:<5}) - Fecha inicio: {formatted_start:<24} - Fecha fin: {formatted_end:<24}")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_sessions_by_year(2024)  # Cambia el año según lo necesites


Año    Localidad            Nombre                         ID     Fecha inicio              Fecha fin                
2024  
  Austin              
    Practice 1         (ID: 9607 ) - Fecha inicio: 18 October 2024, 14:30   - Fecha fin: 18 October 2024, 15:30  
  Marina Bay          
    Race               (ID: 9606 ) - Fecha inicio: 22 September 2024, 09:00 - Fecha fin: 22 September 2024, 11:00
    Qualifying         (ID: 9602 ) - Fecha inicio: 21 September 2024, 10:00 - Fecha fin: 21 September 2024, 11:00
    Practice 3         (ID: 9601 ) - Fecha inicio: 21 September 2024, 06:30 - Fecha fin: 21 September 2024, 07:30
    Practice 2         (ID: 9600 ) - Fecha inicio: 20 September 2024, 10:00 - Fecha fin: 20 September 2024, 11:00
    Practice 1         (ID: 9599 ) - Fecha inicio: 20 September 2024, 06:30 - Fecha fin: 20 September 2024, 07:30
  Baku                
    Race               (ID: 9598 ) - Fecha inicio: 15 September 2024, 08:00 - Fecha fin: 15 September 2024, 10:00
    Qual

# Localidades por año

In [69]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz

# Función para obtener localidades por año
def get_locations_by_year(year):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    # Definir la zona horaria de Argentina (GMT-3)
    tz = pytz.timezone('America/Argentina/Buenos_Aires')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Usar un diccionario para almacenar las localidades y las fechas de las sesiones
        locations = {}

        # Recorrer las sesiones y añadir las localidades y fechas al diccionario
        for session in data_sessions:
            location = session['location']
            # Convertir la fecha de inicio a la zona horaria deseada
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00")).astimezone(tz)
            formatted_date = date_start.strftime("%d %B %Y, %H:%M")

            # Añadir la localidad y la fecha al diccionario
            locations[location] = formatted_date

        # Imprimir encabezado
        print(f"Localidades donde se corrió en {year}:\n")
        print(f"{'Localidad':<20} {'Fecha de Carrera':<30}")
        print("-" * 50)  # Línea de separación

        # Añadir las filas
        for location in sorted(locations):
            print(f"{location:<20} {locations[location]:<30}")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_locations_by_year(2024)  # Cambia el año según lo necesites


Localidades donde se corrió en 2024:

Localidad            Fecha de Carrera              
--------------------------------------------------
Austin               18 October 2024, 14:30        
Baku                 15 September 2024, 08:00      
Barcelona            23 June 2024, 10:00           
Budapest             21 July 2024, 10:00           
Imola                19 May 2024, 10:00            
Jeddah               09 March 2024, 14:00          
Marina Bay           22 September 2024, 09:00      
Melbourne            24 March 2024, 01:00          
Miami                05 May 2024, 17:00            
Monaco               26 May 2024, 10:00            
Montréal             09 June 2024, 15:00           
Monza                01 September 2024, 10:00      
Sakhir               02 March 2024, 12:00          
Shanghai             21 April 2024, 04:00          
Silverstone          07 July 2024, 11:00           
Spa-Francorchamps    28 July 2024, 10:00           
Spielberg            30 Jun

# Sesiones por año y localidad

In [70]:
from urllib.request import urlopen, Request
import json
import os
from datetime import datetime
import pytz  # Importar la librería pytz

# Función para obtener sesiones por año y localidad
def get_sessions_by_year_and_location(year, location):
    # Define la URL de la API para obtener todas las sesiones
    url = f'https://api.openf1.org/v1/sessions?year={year}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        data_sessions = json.loads(data)

        # Filtrar las sesiones por localidad
        filtered_sessions = [session for session in data_sessions if session['location'].lower() == location.lower()]

        # Ordenar las sesiones por date_start de más nueva a más antigua
        filtered_sessions.sort(key=lambda x: x['date_start'], reverse=True)

        # Imprimir la información de las sesiones
        current_year = None
        current_location = None
        print(f"{'Año':<6} {'Localidad':<20} {'Nombre':<30} {'ID':<6} {'Fecha inicio':<25} {'Fecha fin':<25}")
        print("=" * 120)

        # Definir la zona horaria de Argentina (GMT-3)
        tz = pytz.timezone('America/Argentina/Buenos_Aires')

        for session in filtered_sessions:
            if session['year'] != current_year:
                current_year = session['year']
                print(f"{current_year:<6}")

            if session['location'] != current_location:
                current_location = session['location']
                print(f"  {current_location:<20}")

            # Convertir las fechas a tu zona horaria
            date_start = datetime.fromisoformat(session['date_start'].replace("Z", "+00:00"))
            date_end = datetime.fromisoformat(session['date_end'].replace("Z", "+00:00"))

            # Ajustar a la zona horaria GMT-3
            date_start = date_start.astimezone(tz)
            date_end = date_end.astimezone(tz)

            # Formatear las fechas
            formatted_start = date_start.strftime("%d %B %Y, %H:%M")
            formatted_end = date_end.strftime("%d %B %Y, %H:%M")

            # Imprimir nombre de sesión, ID y fechas con ajuste de espacio
            print(f"    {session['session_name']:<18} (ID: {session['session_key']:<5}) - Fecha inicio: {formatted_start:<24} - Fecha fin: {formatted_end:<24}")

        # Si no hay sesiones filtradas, informar al usuario
        if not filtered_sessions:
            print(f"No se encontraron sesiones en {location} para el año {year}.")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso
get_sessions_by_year_and_location(2024, 'Baku')  # Cambia el año y la localidad según lo necesites


Año    Localidad            Nombre                         ID     Fecha inicio              Fecha fin                
2024  
  Baku                
    Race               (ID: 9598 ) - Fecha inicio: 15 September 2024, 08:00 - Fecha fin: 15 September 2024, 10:00
    Qualifying         (ID: 9594 ) - Fecha inicio: 14 September 2024, 09:00 - Fecha fin: 14 September 2024, 10:00
    Practice 3         (ID: 9593 ) - Fecha inicio: 14 September 2024, 05:30 - Fecha fin: 14 September 2024, 06:30
    Practice 2         (ID: 9592 ) - Fecha inicio: 13 September 2024, 10:00 - Fecha fin: 13 September 2024, 11:00
    Practice 1         (ID: 9591 ) - Fecha inicio: 13 September 2024, 06:30 - Fecha fin: 13 September 2024, 07:30


# Listado de todos los pilotos de una sesion

In [71]:
from urllib.request import urlopen, Request
import json
import os

# Función para obtener pilotos de una sesión específica
def get_drivers_by_session(session_key):
    # Define la URL de la API para obtener todos los pilotos en la sesión especificada
    url = f'https://api.openf1.org/v1/drivers?session_key={session_key}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = json.loads(response.read().decode('utf-8'))

        # Imprimir la información de los pilotos
        print(f"{'Número':<10} {'Nombre completo':<30} {'Equipo':<20} {'País':<5}")
        print("=" * 80)

        for driver in data:
            print(f"{driver['driver_number']:<10} {driver['full_name']:<30} {driver['team_name']:<20} {driver['country_code']}")

    except Exception as e:
        print("An error occurred:", str(e))

get_drivers_by_session(9598)  # Reemplaza 9158 con el ID de la sesión deseada



Número     Nombre completo                Equipo               País 
1          Max VERSTAPPEN                 Red Bull Racing      NED
3          Daniel RICCIARDO               RB                   AUS
4          Lando NORRIS                   McLaren              GBR
10         Pierre GASLY                   Alpine               FRA
11         Sergio PEREZ                   Red Bull Racing      MEX
14         Fernando ALONSO                Aston Martin         ESP
16         Charles LECLERC                Ferrari              MON
18         Lance STROLL                   Aston Martin         CAN
22         Yuki TSUNODA                   RB                   JPN
23         Alexander ALBON                Williams             THA
24         ZHOU Guanyu                    Kick Sauber          CHN
27         Nico HULKENBERG                Haas F1 Team         GER
31         Esteban OCON                   Alpine               FRA
43         Franco COLAPINTO               Williams          

# Informacion de un Piloto a partir de su drive_number

In [73]:
from urllib.request import urlopen, Request
import json
import os

# Función para obtener información de un piloto en una sesión específica
def get_driver_info_by_session(driver_number, session_key):
    # Define la URL de la API para obtener información del piloto en una sesión específica
    url = f'https://api.openf1.org/v1/drivers?driver_number={driver_number}&session_key={session_key}'
    api_key = os.getenv('OpenF1')  # Asegúrate de que la clave API esté en la variable de entorno

    # Configura la solicitud con la clave API en los encabezados
    request = Request(url)
    request.add_header('Authorization', f'Bearer {api_key}')

    try:
        # Realiza la solicitud a la API
        response = urlopen(request)
        data = response.read().decode('utf-8')

        # Cargar la respuesta como JSON
        driver_data = json.loads(data)

        # Verificar si se encontraron resultados
        if driver_data:
            for driver in driver_data:  # Si hay varios resultados, los imprimimos
                print(f"Nombre completo: {driver['full_name']}")
                print(f"Nombre en transmisión: {driver['broadcast_name']}")
                print(f"Número de piloto: {driver['driver_number']}")
                print(f"Equipo: {driver['team_name']}")
                print(f"Código de país: {driver['country_code']}")
                print(f"Nombre abreviado: {driver['name_acronym']}")
                print(f"Color del equipo: {driver['team_colour']}")
                print(f"URL de foto: {driver['headshot_url']}")
                print(f"ID de reunión: {driver['meeting_key']}")
                print(f"ID de sesión: {driver['session_key']}")
                print("-" * 50)  # Separador
        else:
            print("No se encontró información para el piloto o sesión proporcionados.")

    except Exception as e:
        print("An error occurred:", str(e))

# Ejemplo de uso

driver_number=43 #Franco Colapinto
session_key=9598
get_driver_info_by_session(driver_number, session_key)  # Cambia el número del piloto y la clave de la sesión según lo necesites


Nombre completo: Franco COLAPINTO
Nombre en transmisión: F COLAPINTO
Número de piloto: 43
Equipo: Williams
Código de país: ARG
Nombre abreviado: COL
Color del equipo: 64C4FF
URL de foto: None
ID de reunión: 1245
ID de sesión: 9598
--------------------------------------------------
